# UK Power Networks Power Cuts - API Exploration

**Goal:** Fetch and explore live power outage data from UK Power Network. 
## API Endpoint <br>  
**CKAN DataStore API:**  https://spenergynetworks.opendatasoft.com/api/explore/v2.1/catalog/datasets/distribution-network-live-outages/records

## 0. Setup And Imports

In [5]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()
UKPN_API_KEY = os.getenv('UKPN_API_KEY')

# Display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 1. API Configuration

In [6]:
# API Configuration
BASE_URL = "https://ukpowernetworks.opendatasoft.com/api/explore/v2.1"
DATASET_ID = "ukpn-live-faults"
API_ENDPOINT = f"{BASE_URL}/catalog/datasets/{DATASET_ID}/records"

headers = {"Authorization": f"Apikey {UKPN_API_KEY}"}
params = {"limit": 100, "timezone": "Europe/London"}

## 2. Dataset Metadata

In [7]:
# Metadata was defined on the page where the dataset is hosted
print("DATASET: UK Power Networks - Live Faults")
print("=" * 80)
print("Dataset ID:", DATASET_ID)
print("Coverage: EPN, LPN, SPN (London, Eastern, South Eastern)")
print("Privacy: Incidents affecting ≤5 customers omitted\n")

DATASET: UK Power Networks - Live Faults
Dataset ID: ukpn-live-faults
Coverage: EPN, LPN, SPN (London, Eastern, South Eastern)
Privacy: Incidents affecting ≤5 customers omitted



## 3. Fetch Data

In [9]:
response = requests.get(API_ENDPOINT, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    total_count = data.get('total_count', 0)

    # Convert to DataFrame - try different structures
    results = data.get('results', [])
    try:
        df = pd.DataFrame([r['fields'] for r in results])
    except KeyError:
        try:
            df = pd.DataFrame([r['record']['fields'] for r in results])
        except KeyError:
            df = pd.DataFrame(results)

    # Display Schema Overview
    print("=" * 80)
    print("SCHEMA OVERVIEW")
    print("=" * 80)
    print(f"Total Records Available: {total_count}")
    print(f"Records Retrieved: {len(df)}")
    print(f"Columns: {len(df.columns)}\n")

    # Display Column Datatypes
    print("-" * 80)
    print("COLUMN DATATYPES")
    print("-" * 80)
    display(df.dtypes)

    # Display Sample Data
    print("\n" + "=" * 80)
    print("SAMPLE DATA (First 5 Rows)")
    print("=" * 80)
    display(df.head(2))

    # Key Columns Summary
    print("\n" + "-" * 80)
    print("KEY COLUMNS:")
    print("-" * 80)
    print("✓ incidentreference - Unique incident ID")
    print("✓ powercuttype - Planned/Unplanned/Restored")
    print("✓ nocustomeraffected - Number of customers affected")
    print("✓ postcodesaffected - Affected postcode sectors")
    print("✓ estimatedrestorationdate - Estimated restoration time")
    print("✓ geopoint - {'lon': x, 'lat': y} geographic coordinates")
    print("✓ operatingzone - UK Power Networks operational zone")
    print("=" * 80)

else:
    print(f"✗ Error {response.status_code}: {response.text}")

# =======================
# Check unique values in powercuttype column
print("\n" + "=" * 80)
print("UNIQUE VALUES ANALYSIS")
print("=" * 80)

if 'powercuttype' in df.columns:
    print(f"\nUnique values in 'powercuttype' column:")
    print(df['powercuttype'].unique())
    print(f"\nValue counts:")
    print(df['powercuttype'].value_counts().to_dict())
    print(f"\nData type: {df['powercuttype'].dtype}")
else:
    print("\n⚠️ 'powercuttype' column not found in dataframe")

# Check a few sample values
print(f"\nSample powercuttype values:")
print(df[['powercuttype']].head(10))

SCHEMA OVERVIEW
Total Records Available: 215
Records Retrieved: 100
Columns: 29

--------------------------------------------------------------------------------
COLUMN DATATYPES
--------------------------------------------------------------------------------


incidentreference                              object
powercuttype                                   object
creationdatetime                               object
nocallsreported                                 int64
incidentscount                                  int64
nocustomeraffected                              int64
postcodesaffected                              object
restoredincidents                              object
unplannedincidents                             object
plannedincidents                               object
incidenttypetbcestimatedfriendlydescription    object
incidentdescription                            object
fullpostcodedata                               object
incidentcategorycustomerfriendlydescription    object
incidentcategory                               object
incidenttypename                               object
incidenttype                                    int64
incidentpriority                                int64
statusid                    


SAMPLE DATA (First 5 Rows)


,incidentreference,powercuttype,creationdatetime,nocallsreported,incidentscount,nocustomeraffected,postcodesaffected,restoredincidents,unplannedincidents,plannedincidents,incidenttypetbcestimatedfriendlydescription,incidentdescription,fullpostcodedata,incidentcategorycustomerfriendlydescription,incidentcategory,incidenttypename,incidenttype,incidentpriority,statusid,restoreddatetime,planneddate,receiveddate,noplannedcustomers,plannedincidentreason,message,mainmessage,geopoint,estimatedrestorationdate,operatingzone
0,INCD-104738-C,Planned,2025-10-04T16:07:29,11,0,0,IP22 2;IP22 3;IP22 5,None,None,None,20 Nov 00:30 - 01:30,None,IP222AA;IP222AT;IP222AB;IP225XN,"We regularly inspect our electrical equipment to ensure it is safe and efficient. We need to examine three transformers on our electricity poles to check they are functioning properly. This essential work helps to provide a reliable electricity supply to your property, reducing the risk of any unplanned power cuts in the future. To keep everyone safe while we carry out these works, we will need to temporarily turn off your electricity. We are taking extra measures to limit the time your power is off and we'll do our best to cause as little disruption as possible. Doing this will reduce the risk of you having an unplanned power cut in the future as we appreciate your area has been impacted with unplanned power cuts in the last 6 months. This work is taking place in the Freezen Hill, Blue Pump Farm and Hill Farm area.",28,Planned,3,12,5,2025-11-20T01:13:01.447,2025-11-19T23:00:00,2025-11-19T23:00:00,41,"We regularly inspect our electrical equipment to ensure it is safe and efficient. We need to examine three transformers on our electricity poles to check they are functioning properly. This essential work helps to provide a reliable electricity supply to your property, reducing the risk of any unplanned power cuts in the future. To keep everyone safe while we carry out these works, we will need to temporarily turn off your electricity. We are taking extra measures to limit the time your power is off and we'll do our best to cause as little disruption as possible. Doing this will reduce the risk of you having an unplanned power cut in the future as we appreciate your area has been impacted with unplanned power cuts in the last 6 months. This work is taking place in the Freezen Hill, Blue Pump Farm and Hill Farm area.",None,The electricity supply to your premises may currently be affected by planned work. You should have received a letter explaining this. We hope to be able to restore your supplies by 20-NOV-2025 01:00. Please accept our sincere apologies for any inconvenience this is causing.,"{'lon': 0.99109, 'lat': 52.38511}",2025-11-20T01:00:00,BURY ST EDMUNDS
1,INCD-105605-C,Planned,2025-11-17T12:34:16,11,0,0,HP23 4,None,None,None,20 Nov 14:30 - 15:30,None,HP234NT;HP234PA;HP234NZ,"To provide a reliable electricity supply to your property, we carry out regular inspections of trees near our overhead cables. This has identified trees that have grown close, which could damage the cables and affect your electricity supply. Our engineers need to temporarily switch off your power so we can safely cut the trees away from the cables. We are aware that the area has been affected by 7 unplanned power cuts in the last 6 months and doing this will reduce the risk of you having an unplanned power cut in the future. This work is taking place on The Moors.",28,Planned,2,12,1,None,2025-11-20T09:00:00,2025-11-20T09:00:00,0,"To provide a reliable electricity supply to your property, we carry out regular inspections of trees near our overhead cables. This has identified trees that have grown close, which could damage the cables and affect your electricity supply. Our engineers need to temporarily switch off your power so we can safely cut the trees away from the cables. We are aware that the area has been affected by 7 unplanned power cuts in the last 6 months and doing this will reduce the ri


--------------------------------------------------------------------------------
KEY COLUMNS:
--------------------------------------------------------------------------------
✓ incidentreference - Unique incident ID
✓ powercuttype - Planned/Unplanned/Restored
✓ nocustomeraffected - Number of customers affected
✓ postcodesaffected - Affected postcode sectors
✓ estimatedrestorationdate - Estimated restoration time
✓ geopoint - {'lon': x, 'lat': y} geographic coordinates
✓ operatingzone - UK Power Networks operational zone

UNIQUE VALUES ANALYSIS

Unique values in 'powercuttype' column:
['Planned' 'Unplanned' 'Restored' 'Multiple']

Value counts:
{'Restored': 39, 'Planned': 28, 'Unplanned': 27, 'Multiple': 6}

Data type: object

Sample powercuttype values:
  powercuttype
0      Planned
1      Planned
2      Planned
3      Planned
4      Planned
5      Planned
6      Planned
7      Planned
8      Planned
9      Planned


In [ ]:
# Get dataset metadata
metadata_url = f"{BASE_URL}/catalog/datasets/{DATASET_ID}"
response = requests.get(metadata_url, headers=headers)

if response.status_code == 200:
    metadata = response.json()

    # Look for field descriptions
    if 'fields' in metadata:
        print("=" * 80)
        print("FIELD DEFINITIONS")
        print("=" * 80)
        for field in metadata['fields']:
            if field.get('name') == 'powercuttype':
                print(f"\nField: {field.get('name')}")
                print(f"Label: {field.get('label')}")
                print(f"Description: {field.get('description')}")
                print(f"Type: {field.get('type')}")
                if 'annotations' in field:
                    print(f"Annotations: {field.get('annotations')}")